<a href="https://colab.research.google.com/github/Hamza-Ali0237/PyTorch-GAN-Implementation/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing GAN from scratch in PyTorch

Dataset: [https://www.kaggle.com/datasets/kvpratama/pokemon-images-dataset](https://www.kaggle.com/datasets/kvpratama/pokemon-images-dataset)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch as t
import torch.nn as nn
import torchvision as tv
import matplotlib.pyplot as plt

In [ ]:
# Deep Convolutional Generator Block
def dc_gen_block(in_dim, out_dim, kernel_size, stride):
  return nn.Sequential(
      nn.ConvTranspose2d(in_dim, out_dim, kernel_size, stride=stride),
      nn.BatchNorm2d(out_dim),
      nn.ReLU()
  )

# Generator Loss Function
def gen_loss(gen, disc, num_images, z_dim):
  noise = t.randn(num_images, z_dim)
  fake = gen(noise)
  disc_pred = disc(fake)
  criterion = nn.BCEWithLogitsLoss()
  gen_loss = criterion(disc_pred, t.ones_like(disc_pred))
  return gen_loss

# Deep Convolutional Generator Class
class DCGenerator(nn.Module):
  def __init__(self, in_dim, kernel_size=4, stride=2):
    super(DCGenerator, self).__init__()
    self.in_dim = in_dim
    self.gen = nn.Sequential(
        dc_gen_block(in_dim, 1024, kernel_size, stride),
        dc_gen_block(1024, 512, kernel_size, stride),
        dc_gen_block(512, 256, kernel_size, stride),
        nn.ConvTranspose2d(256, 3, kernel_size, stride=stride),
        nn.Tanh()
    )

  def forward(self, x):
    x = x.view(len(x), self.in_dim, 1, 1)
    return self.gen(x)

In [ ]:
# Deep Convolutional Discriminator Block
def dc_disc_block(in_dim, out_dim, kernel_size, stride):
  return nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size, stride=stride),
      nn.BatchNorm2d(out_dim),
      nn.LeakyReLU(0.2)
  )

# Discriminator Loss Function
def disc_loss(gen, disc, real, num_images, z_dim):
  criterion = BCEWithLogitsLoss()
  noise = t.randn(num_images, z_dim)
  fake = gen(noise)
  disc_pred_fake = disc(fake)
  fake_loss = criterion(disc_pred_fake, t.zeros_like(disc_pred_fake))
  disc_pred_real = disc(real)
  real_loss = criterion(disc_pred_real, t.zeros_like(disc_pred_real))
  disc_loss (real_loss + fake_loss) / 2
  return disc_loss

# Deep Convolutional Discriminator Class
class DCDiscriminator(nn.Module):
  def __init__(self, kernel_size=4, stride=2):
    super(DCDiscriminator, self).__init__()
    self.disc = nn.Sequential(
        dc_disc_block(3, 512, kernel_size, stride),
        dc_disc_block(512, 1024, kernel_size, stride),
        nn.Conv2d(1024, 1, kernel_size, stride)
    )

  def forward(self, x):
    x = self.disc(x)
    return x.view(len(x), -1)

# Loading Dataset

In [ ]:
# Define Transformations
transform = tv.transforms.Compose([
    tv.transforms.Resize(64),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = tv.datasets.ImageFolder(root="/content/drive/MyDrive/pokemon-dataset/pokemon", transform=transform)

dataloader_train = t.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

# Training Model

In [ ]:
# Training Loop
z_dim = 100
lr = 0.0002
batch_size = 128
EPOCHS = 10

for epoch in range(EPOCHS):
  for real in dataloader_train:
    cur_batch_size = len(real)

    disc_opt.zero_grad()
    disc_loss = disc_loss(gen, disc, real, cur_batch_size, z_dim=16)
    disc_loss.backward()
    disc_opt.step()

    gen_opt.zero_grad()
    gen_loss = gen_loss(gen, disc, cur_batch_size, z_dim=16)
    gen_loss.backward()
    gen_opt.step()